In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [156]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# Step 1: Read the data
train_data = pd.read_csv('/content/drive/MyDrive/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')


In [157]:
#seperate cihaz301 and cihaz305
train_301=train_data[train_data["SERVER"]=="CIHAZ_301"]
train_305=train_data[train_data["SERVER"]=="CIHAZ_305"]
test_301=test_data[test_data["SERVER"]=="CIHAZ_301"]
test_305=test_data[test_data["SERVER"]=="CIHAZ_305"]

In [158]:
#get mean of all cpus according to datetimes
train_301=train_301.groupby("DATETIME")["CPULOAD"].mean().to_frame().reset_index()
train_305=train_305.groupby("DATETIME")["CPULOAD"].mean().to_frame().reset_index()


In [159]:
#datatime object
train_301['DATETIME'] = pd.to_datetime(train_301['DATETIME'])
train_305['DATETIME'] = pd.to_datetime(train_305['DATETIME'])
test_301['DATETIME'] = pd.to_datetime(test_301['DATETIME'])
test_305['DATETIME'] = pd.to_datetime(test_305['DATETIME'])

<ipython-input-159-3e2bd50e4ef9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_301['DATETIME'] = pd.to_datetime(test_301['DATETIME'])
<ipython-input-159-3e2bd50e4ef9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_305['DATETIME'] = pd.to_datetime(test_305['DATETIME'])


In [167]:
#extract time zones from dataframe
train_301['DATETIME'] = train_301['DATETIME'].dt.tz_localize(None)
train_305['DATETIME'] = train_305['DATETIME'].dt.tz_localize(None)
test_301['DATETIME'] = test_301['DATETIME'].dt.tz_localize(None)
test_305['DATETIME'] = test_305['DATETIME'].dt.tz_localize(None)


<ipython-input-167-88be5b9ef0dc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_301['DATETIME'] = test_301['DATETIME'].dt.tz_localize(None)
<ipython-input-167-88be5b9ef0dc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_305['DATETIME'] = test_305['DATETIME'].dt.tz_localize(None)


In [169]:
#prepare train data for prophet
train_301_p = train_301.rename(columns={'DATETIME': 'ds', 'CPULOAD': 'y'})
train_301_p = train_301_p[["ds","y"]]
train_301_p.tail()

train_305_p = train_305.rename(columns={'DATETIME': 'ds', 'CPULOAD': 'y'})
train_305_p = train_305_p[["ds","y"]]
train_305_p.tail()

,ds,y
6509,2020-05-30 22:45:00,13.932917
6510,2020-05-30 23:00:00,10.920833
6511,2020-05-30 23:15:00,10.177917
6512,2020-05-30 23:30:00,9.696667
6513,2020-05-30 23:45:00,9.147917


In [170]:
# prpohet for server 301
model1 = Prophet()
model1.fit(train_301_p)


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt63qy7ub/ji7pe47l.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt63qy7ub/ewnjh5xs.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=301', 'data', 'file=/tmp/tmpt63qy7ub/ji7pe47l.json', 'init=/tmp/tmpt63qy7ub/ewnjh5xs.json', 'output', 'file=/tmp/tmpt63qy7ub/prophet_modelf4amcew6/prophet_model-20230713121633.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:16:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:16:34 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [171]:
# prpohet for server 305
model2 = Prophet()
model2.fit(train_305_p)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt63qy7ub/y8j7yui0.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt63qy7ub/mkfuhu07.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=40457', 'data', 'file=/tmp/tmpt63qy7ub/y8j7yui0.json', 'init=/tmp/tmpt63qy7ub/mkfuhu07.json', 'output', 'file=/tmp/tmpt63qy7ub/prophet_modelksxicusi/prophet_model-20230713121640.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:16:40 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:16:40 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [172]:
#prepare test data for prophet
test_301_p = test_301.rename(columns={'DATETIME': 'ds'})
test_301_p = test_301_p[["ds"]]
test_301_p.tail()
test_305_p = test_305.rename(columns={'DATETIME': 'ds'})
test_305_p = test_305_p[["ds"]]
test_305_p.tail()

,ds
183,2020-05-31 22:45:00
185,2020-05-31 23:00:00
187,2020-05-31 23:15:00
189,2020-05-31 23:30:00
191,2020-05-31 23:45:00


In [173]:
#predict values for server 301 and server 305
forecast_server_301 = model1.predict(test_301_p)['yhat'].values
forecast_server_305 = model2.predict(test_305_p)['yhat'].values

In [174]:
#generate dataframes
test_301['forecast']=forecast_server_301
test_305['forecast']=forecast_server_305

<ipython-input-174-347e7205b5df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_301['forecast']=forecast_server_301
<ipython-input-174-347e7205b5df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_305['forecast']=forecast_server_305


In [176]:
#merge predictions for server 301 and server 305
test = pd.concat([test_301, test_305], axis=0)

In [177]:
#sort for datatime
test = test.sort_values('DATETIME')

In [185]:
test

,index,DATETIME,SERVER,forecast
0,0,2020-05-31 00:00:00,CIHAZ_301,4.964389
1,1,2020-05-31 00:00:00,CIHAZ_305,7.494997
3,3,2020-05-31 00:15:00,CIHAZ_305,6.938765
2,2,2020-05-31 00:15:00,CIHAZ_301,4.684828
5,5,2020-05-31 00:30:00,CIHAZ_305,6.420511
...,...,...,...,...
186,186,2020-05-31 23:15:00,CIHAZ_301,6.936978
189,189,2020-05-31 23:30:00,CIHAZ_305,11.256976
188,188,2020-05-31 23:30:00,CIHAZ_301,6.660718
190,190,2020-05-31 23:45:00,CIHAZ_301,6.384443


In [201]:
sample_submission = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [203]:
sample_submission['CPULOAD']=test['forecast']

In [204]:
sample_submission

,index,CPULOAD
0,0,4.964389
1,1,7.494997
2,2,4.684828
3,3,6.938765
4,4,4.418803
...,...,...
187,187,11.832721
188,188,6.660718
189,189,11.256976
190,190,6.384443


In [205]:
sample_submission.to_csv("/content/drive/MyDrive/sample_submission.csv",index=False)

In [206]:
sample_submission

,index,CPULOAD
0,0,4.964389
1,1,7.494997
2,2,4.684828
3,3,6.938765
4,4,4.418803
...,...,...
187,187,11.832721
188,188,6.660718
189,189,11.256976
190,190,6.384443
